<a href="https://colab.research.google.com/github/Cloud-Course-Group-Phoenix/Project-Pheonix/blob/main/Fake_Data_Generator_for_Cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install paho-mqtt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 2.7 MB/s eta 0:00:00


In [3]:
# generate fake sensor data because the broler doesn't work
import time
import json
import random as rand
import paho.mqtt.client as mqtt

def generate_fake_sensor_data():
    """
    Generates fake sensor data for Temperature, Humidity, and Distance.

    Returns:
        dict: A dictionary containing the sensor data.
    """
    temperature = round(rand.uniform(15, 35), 1)  # Temperature between 15 and 35 degrees Celsius
    humidity = round(rand.uniform(30, 80), 1)     # Humidity between 30 and 80 percent
    distance = round(rand.uniform(10, 100), 1)   # Distance between 10 and 100 cm
    return {
        "Temperature": temperature,
        "Humidity": humidity,
        "Distance": distance,
    }

def create_mqtt_message(sensor_data, topic="braude/D106/indoor"):
    """
    Creates an MQTT message with a topic and JSON payload.

    Args:
        sensor_data (dict): The sensor data dictionary.
        topic (str, optional): The MQTT topic. Defaults to "braude/D106/indoor".

    Returns:
        tuple: A tuple containing the topic and the JSON payload string.
    """
    payload = json.dumps(sensor_data)  # Convert the dictionary to a JSON string
    return topic, payload

def on_connect(client, userdata, flags, rc):
    """Callback function for when the client connects to the broker."""
    if rc == 0:
        print("Connected to MQTT Broker (HiveMQ)!")
    else:
        print(f"Failed to connect, return code {rc}")

def on_disconnect(client, userdata, rc):
    """Callback function for when the client disconnects from the broker."""
    if rc != 0:
        print(f"Unexpected disconnection with code {rc}")


# Example of how to use the functions in a Jupyter cell:
broker_address = "broker.hivemq.com"
port = 1883
client = mqtt.Client()
client.on_connect = on_connect
client.on_disconnect = on_disconnect
try:
    client.connect(broker_address, port)
except Exception as e:
    print(f"Error connecting to broker: {e}")
client.loop_start()

try:
    while(True):  # Send 5 messages as an example
        sensor_data = generate_fake_sensor_data()
        topic, payload = create_mqtt_message(sensor_data)
        client.publish(topic, payload)
        print(f"Published to {topic}: {payload}")
        time.sleep(1)
except KeyboardInterrupt:
    print("Interrupted")
finally:
    client.disconnect()
    client.loop_stop()

<ipython-input-3-73b445564afd>:53: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Published to braude/D106/indoor: {"Temperature": 26.9, "Humidity": 74.3, "Distance": 41.8}
Published to braude/D106/indoor: {"Temperature": 34.2, "Humidity": 62.9, "Distance": 41.0}
Published to braude/D106/indoor: {"Temperature": 27.3, "Humidity": 63.5, "Distance": 10.2}
Published to braude/D106/indoor: {"Temperature": 28.5, "Humidity": 50.0, "Distance": 41.6}
Published to braude/D106/indoor: {"Temperature": 27.1, "Humidity": 70.4, "Distance": 35.9}
Published to braude/D106/indoor: {"Temperature": 28.3, "Humidity": 51.4, "Distance": 59.3}
Published to braude/D106/indoor: {"Temperature": 32.9, "Humidity": 69.6, "Distance": 34.9}
Published to braude/D106/indoor: {"Temperature": 29.5, "Humidity": 41.2, "Distance": 93.8}
Connected to MQTT Broker (HiveMQ)!
Published to braude/D106/indoor: {"Temperature": 33.2, "Humidity": 40.0, "Distance": 16.4}
Published to braude/D106/indoor: {"Temperature": 32.4, "Humidity": 51.7, "Distance": 96.8}
Published to braude/D106/indoor: {"Temperature": 29.7, 